In [2]:
import os

from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken
from autogen_agentchat.base import TaskResult

from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console


from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat

In [3]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.environ["GITHUB_TOKEN"]),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

c:\Users\Me\anaconda3\envs\ai_agents\Lib\site-packages\autogen_ext\models\azure\_azure_ai_client.py:307: UserWarning: Missing required field 'structured_output' in ModelInfo. This field will be required in a future version of AutoGen.
  validate_model_info(config["model_info"])


In [4]:
# Renamed + repurposed from your originals

triage_agent = AssistantAgent(
    "triage_agent",
    model_client=client,
    description="Triage assistant for customer service. Collects essentials and proposes exactly one next action.",
    system_message="""
You are a Customer Service Triage Agent with ten years of experience. You are brief and solution-first.
Goal: collect the key details (issue, user/account, urgency) and propose ONE concrete next step.

STRICT RULES:
- Output MUST be valid JSON with this exact shape:
  {
    "issue": "<plain-language summary in one sentence>",
    "user_id": "<string or null if unknown>",
    "urgency": "<low|medium|high>",
    "next_action": "<the single best next action>",
    "user_reply": "<ONE short sentence to send to the customer>"
  }
- Only ONE actionable recommendation in "next_action".
- Be concise. No small talk.
- If user seems upset, the "user_reply" may include ONE brief empathy clause (still one sentence).

EXAMPLES OF 'next_action':
- "Verify order number and request a photo of the damaged item."
- "Offer a prepaid return label and create RMA."
- "Check shipment status and provide updated ETA."

If information is missing, set user_id to null and choose the best next step to obtain it.
""",
)

concierge_qa_agent = AssistantAgent(
    "concierge_qa_agent",
    model_client=client,
    description="Quality gate for CS replies: checks policy, clarity, tone, and whether the next step is the single best one.",
    system_message="""
You are a Customer Service QA Concierge. You insist on local, authentic customer experience and clear policy adherence.

Goal: Decide if the triage_agent's output is the BEST single next step for the customer right now.

STRICT OUTPUT:
- If acceptable, respond with EXACTLY: APPROVE
- If not, respond with: REFINE: <guidance in <=140 characters, no examples, no long text>

CHECKS:
- Single best next step? (no alternatives or lists)
- Tone: brief, helpful, on-brand, optional short empathy if needed
- Policy alignment (returns/warranty/refunds/SLAs)
- Clarity and actionability

Do NOT provide a specific rewritten reply—only guidance if refinement is needed.
""",
)


In [5]:
# Terminate when the QA (concierge_qa_agent) responds with "APPROVE"
termination = TextMentionTermination("APPROVE")

# Use the new minimal-role agents
team = RoundRobinGroupChat(
    [triage_agent, concierge_qa_agent],
    termination_condition=termination
)

# Example CS task (replace with your runtime input)
task = "Customer says: 'My order #A123 arrived damaged and I need a replacement.'"

# Stream results until termination hits ("APPROVE") or the run ends
async for message in team.run_stream(task=task):
    if isinstance(message, TaskResult):
        print("Stop Reason:", message.stop_reason)   # e.g., "termination_condition_met"
    else:
        print(message)


id='f0f7d985-ae12-4157-9dd8-a7e0f90bc011' source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 8, 12, 2, 4, 26, 912869, tzinfo=datetime.timezone.utc) content="Customer says: 'My order #A123 arrived damaged and I need a replacement.'" type='TextMessage'
id='6651d4a3-cf00-4991-bbfc-eeb981b7f504' source='triage_agent' models_usage=RequestUsage(prompt_tokens=256, completion_tokens=72) metadata={} created_at=datetime.datetime(2025, 8, 12, 2, 4, 29, 171831, tzinfo=datetime.timezone.utc) content='{\n  "issue": "Order A123 arrived damaged and requires a replacement.",\n  "user_id": null,\n  "urgency": "high",\n  "next_action": "Offer a prepaid return label and create RMA.",\n  "user_reply": "I\'m sorry to hear your order arrived damaged; I\'ll arrange for a replacement right away."\n}' type='TextMessage'
id='9c93e2fb-086e-458c-ab01-05a3c9b2ce60' source='concierge_qa_agent' models_usage=RequestUsage(prompt_tokens=253, completion_tokens=18) metadata={} created_at=dateti